In [4]:
a= list(range(10))
a

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [5]:
del(a)

In [6]:
a[0]a

NameError: name 'a' is not defined

In [1]:
label_day = "123333"
print(label_day,"not buy size befor duplicate")

123333 not buy size befor duplicate


In [3]:
#rule1
P = 0.03768433
F = 0.0005877342
R = F*P/(2*P-F)
R

0.0002961767229830952

In [4]:
#rule2
P = 0.05189621
F = 0.0006846610
R = F*P/(2*P-F)
R

0.0003446036591175452

In [5]:
#rule3
P = 0.04705882
F = 0.0006931702
R = F*P/(2*P-F)
R

0.00034915661540274535

In [ ]:
def get_label_testset(train_user,LabelDay):
    # 测试集选为上一天所有的交互数据
    data_test = train_user[(train_user['daystime'] == LabelDay)]#&((train_user.behavior_type==3)|(train_user.behavior_type==2))
    data_test = data_test.drop_duplicates(['user_id', 'item_id'])
    return data_test[['user_id', 'item_id','item_category']]



def item_category_feture(data,end_time,beforeoneday):
    # data = Data[(Data['daystime']<LabelDay) & (Data['daystime']>LabelDay-datetime.timedelta(days=FEATURE_EXTRACTION_SLOT))]
    item_count = pd.crosstab(data.item_category,data.behavior_type)
    item_count_before5=None
    if (((end_time-datetime.timedelta(days=5))<datetime.datetime(2014,12,13,0,0,0))&((end_time-datetime.timedelta(days=5))>datetime.datetime(2014,12,10,0,0,0))):
        beforefiveday = data[data['daystime']>=end_time-datetime.timedelta(days=5+2)]
        item_count_before5 = pd.crosstab(beforefiveday.item_category,beforefiveday.behavior_type)
    else:
        beforefiveday = data[data['daystime']>=end_time-datetime.timedelta(days=5)]
        item_count_before5 = pd.crosstab(beforefiveday.item_category,beforefiveday.behavior_type)
    item_count_before_3=None
    if (((end_time-datetime.timedelta(days=5))<datetime.datetime(2014,12,13,0,0,0))&((end_time-datetime.timedelta(days=5))>datetime.datetime(2014,12,10,0,0,0))):
        beforethreeday = data[data['daystime']>=end_time-datetime.timedelta(days=3+2)]
        item_count_before_3 = pd.crosstab(beforethreeday.item_category,beforethreeday.behavior_type)
    else:
        beforethreeday = data[data['daystime']>=end_time-datetime.timedelta(days=3)]
        item_count_before_3 = pd.crosstab(beforethreeday.item_category,beforethreeday.behavior_type)

    item_count_before_2=None
    if (((end_time-datetime.timedelta(days=5))<datetime.datetime(2014,12,13,0,0,0))&((end_time-datetime.timedelta(days=5))>datetime.datetime(2014,12,10,0,0,0))):
        beforethreeday = data[data['daystime']>=end_time-datetime.timedelta(days=7+2)]
        item_count_before_2 = pd.crosstab(beforethreeday.item_category,beforethreeday.behavior_type)
    else:
        beforethreeday = data[data['daystime']>=end_time-datetime.timedelta(days=7)]
        item_count_before_2 = pd.crosstab(beforethreeday.item_category,beforethreeday.behavior_type)
        
    # beforeoneday = Data[Data['daystime'] == LabelDay-datetime.timedelta(days=1)]
    beforeonedayitem_count = pd.crosstab(beforeoneday.item_category,beforeoneday.behavior_type)
    countAverage = item_count/FEATURE_EXTRACTION_SLOT
    buyRate = pd.DataFrame()
    buyRate['click'] = item_count[1]/item_count[4]
    buyRate['skim'] = item_count[2]/item_count[4]
    buyRate['collect'] = item_count[3]/item_count[4]
    buyRate.index = item_count.index

    buyRate_2 = pd.DataFrame()
    buyRate_2['click'] = item_count_before5[1]/item_count_before5[4]
    buyRate_2['skim'] = item_count_before5[2]/item_count_before5[4]
    buyRate_2['collect'] = item_count_before5[3]/item_count_before5[4]
    buyRate_2.index = item_count_before5.index

    buyRate_3 = pd.DataFrame()
    buyRate_3['click'] = item_count_before_3[1]/item_count_before_3[4]
    buyRate_3['skim'] = item_count_before_3[2]/item_count_before_3[4]
    buyRate_3['collect'] = item_count_before_3[3]/item_count_before_3[4]
    buyRate_3.index = item_count_before_3.index


    buyRate = buyRate.replace([np.inf, -np.inf], 0)
    buyRate_2 = buyRate_2.replace([np.inf, -np.inf], 0)
    buyRate_3 = buyRate_3.replace([np.inf, -np.inf], 0)
    item_category_feture = pd.merge(item_count,beforeonedayitem_count,how='left',right_index=True,left_index=True)
    item_category_feture = pd.merge(item_category_feture,countAverage,how='left',right_index=True,left_index=True)
    item_category_feture = pd.merge(item_category_feture,buyRate,how='left',right_index=True,left_index=True)
    item_category_feture = pd.merge(item_category_feture,item_count_before5,how='left',right_index=True,left_index=True)
    item_category_feture = pd.merge(item_category_feture,item_count_before_3,how='left',right_index=True,left_index=True)
    item_category_feture = pd.merge(item_category_feture,item_count_before_2,how='left',right_index=True,left_index=True)
#    item_category_feture = pd.merge(item_category_feture,buyRate_2,how='left',right_index=True,left_index=True)
#    item_category_feture = pd.merge(item_category_feture,buyRate_3,how='left',right_index=True,left_index=True)
    item_category_feture.fillna(0,inplace=True)
    return item_category_feture

def item_id_feture(data,end_time,beforeoneday):   
    # data = Data[(Data['daystime']<LabelDay) & (Data['daystime']>LabelDay-datetime.timedelta(days=FEATURE_EXTRACTION_SLOT))]
    item_count = pd.crosstab(data.item_id,data.behavior_type)
    item_count_before5=None
    if (((end_time-datetime.timedelta(days=5))<datetime.datetime(2014,12,13,0,0,0))&((end_time-datetime.timedelta(days=5))>datetime.datetime(2014,12,10,0,0,0))):
        beforefiveday = data[data['daystime']>=end_time-datetime.timedelta(days=5+2)]
        item_count_before5 = pd.crosstab(beforefiveday.item_id,beforefiveday.behavior_type)
    else:
        beforefiveday = data[data['daystime']>=end_time-datetime.timedelta(days=5)]
        item_count_before5 = pd.crosstab(beforefiveday.item_id,beforefiveday.behavior_type)

    item_count_before_3=None
    if (((end_time-datetime.timedelta(days=5))<datetime.datetime(2014,12,13,0,0,0))&((end_time-datetime.timedelta(days=5))>datetime.datetime(2014,12,10,0,0,0))):
        beforethreeday = data[data['daystime']>=end_time-datetime.timedelta(days=3+2)]
        item_count_before_3 = pd.crosstab(beforethreeday.item_id,beforethreeday.behavior_type)
    else:
        beforethreeday = data[data['daystime']>=end_time-datetime.timedelta(days=3)]
        item_count_before_3 = pd.crosstab(beforethreeday.item_id,beforethreeday.behavior_type)

    item_count_before_2=None
    if (((end_time-datetime.timedelta(days=5))<datetime.datetime(2014,12,13,0,0,0))&((end_time-datetime.timedelta(days=5))>datetime.datetime(2014,12,10,0,0,0))):
        beforethreeday = data[data['daystime']>=end_time-datetime.timedelta(days=7+2)]
        item_count_before_2 = pd.crosstab(beforethreeday.item_id,beforethreeday.behavior_type)
    else:
        beforethreeday = data[data['daystime']>=end_time-datetime.timedelta(days=7)]
        item_count_before_2 = pd.crosstab(beforethreeday.item_id,beforethreeday.behavior_type)
        
    item_count_unq = data.groupby(by = ['item_id','behavior_type']).agg({"user_id":lambda x:x.nunique()});item_count_unq = item_count_unq.unstack()
    # beforeoneday = Data[Data['daystime'] == LabelDay-datetime.timedelta(days=1)]
    beforeonedayitem_count = pd.crosstab(beforeoneday.item_id,beforeoneday.behavior_type)
    countAverage = item_count/FEATURE_EXTRACTION_SLOT
    buyRate = pd.DataFrame()
    buyRate['click'] = item_count[1]/item_count[4]
    buyRate['skim'] = item_count[2]/item_count[4]
    buyRate['collect'] = item_count[3]/item_count[4]
    buyRate.index = item_count.index

    buyRate_2 = pd.DataFrame()
    buyRate_2['click'] = item_count_before5[1]/item_count_before5[4]
    buyRate_2['skim'] = item_count_before5[2]/item_count_before5[4]
    buyRate_2['collect'] = item_count_before5[3]/item_count_before5[4]
    buyRate_2.index = item_count_before5.index

    buyRate_3 = pd.DataFrame()
    buyRate_3['click'] = item_count_before_3[1]/item_count_before_3[4]
    buyRate_3['skim'] = item_count_before_3[2]/item_count_before_3[4]
    buyRate_3['collect'] = item_count_before_3[3]/item_count_before_3[4]
    buyRate_3.index = item_count_before_3.index

    buyRate = buyRate.replace([np.inf, -np.inf], 0)
    buyRate_2 = buyRate_2.replace([np.inf, -np.inf], 0)
    buyRate_3 = buyRate_3.replace([np.inf, -np.inf], 0)
    item_id_feture = pd.merge(item_count,beforeonedayitem_count,how='left',right_index=True,left_index=True)
    item_id_feture = pd.merge(item_id_feture,countAverage,how='left',right_index=True,left_index=True)
    item_id_feture = pd.merge(item_id_feture,buyRate,how='left',right_index=True,left_index=True)
    item_id_feture = pd.merge(item_id_feture,item_count_unq,how='left',right_index=True,left_index=True)
    item_id_feture = pd.merge(item_id_feture,item_count_before5,how='left',right_index=True,left_index=True)
    item_id_feture = pd.merge(item_id_feture,item_count_before_3,how='left',right_index=True,left_index=True)
    item_id_feture = pd.merge(item_id_feture,item_count_before_2,how='left',right_index=True,left_index=True)
#    item_id_feture = pd.merge(item_id_feture,buyRate_2,how='left',right_index=True,left_index=True)
#    item_id_feture = pd.merge(item_id_feture,buyRate_3,how='left',right_index=True,left_index=True)
    item_id_feture.fillna(0,inplace=True)
    return item_id_feture


def user_id_feture(data,end_time,beforeoneday):   
    # data = Data[(Data['daystime']<LabelDay) & (Data['daystime']>LabelDay-datetime.timedelta(days=FEATURE_EXTRACTION_SLOT))]
    user_count = pd.crosstab(data.user_id,data.behavior_type)
    user_count_before5=None
    if (((end_time-datetime.timedelta(days=5))<datetime.datetime(2014,12,13,0,0,0))&((end_time-datetime.timedelta(days=5))>datetime.datetime(2014,12,10,0,0,0))):
        beforefiveday = data[data['daystime']>=end_time-datetime.timedelta(days=5+2)]
        user_count_before5 = pd.crosstab(beforefiveday.user_id,beforefiveday.behavior_type)
    else:
        beforefiveday = data[data['daystime']>=end_time-datetime.timedelta(days=5)]
        user_count_before5 = pd.crosstab(beforefiveday.user_id,beforefiveday.behavior_type)

    user_count_before_3=None
    if (((end_time-datetime.timedelta(days=5))<datetime.datetime(2014,12,13,0,0,0))&((end_time-datetime.timedelta(days=5))>datetime.datetime(2014,12,10,0,0,0))):
        beforethreeday = data[data['daystime']>=end_time-datetime.timedelta(days=3+2)]
        user_count_before_3 = pd.crosstab(beforethreeday.user_id,beforethreeday.behavior_type)
    else:
        beforethreeday = data[data['daystime']>=end_time-datetime.timedelta(days=3)]
        user_count_before_3 = pd.crosstab(beforethreeday.user_id,beforethreeday.behavior_type)

    user_count_before_2=None
    if (((end_time-datetime.timedelta(days=5))<datetime.datetime(2014,12,13,0,0,0))&((end_time-datetime.timedelta(days=5))>datetime.datetime(2014,12,10,0,0,0))):
        beforethreeday = data[data['daystime']>=end_time-datetime.timedelta(days=7+2)]
        user_count_before_2 = pd.crosstab(beforethreeday.user_id,beforethreeday.behavior_type)
    else:
        beforethreeday = data[data['daystime']>=end_time-datetime.timedelta(days=7)]
        user_count_before_2 = pd.crosstab(beforethreeday.user_id,beforethreeday.behavior_type)
        
    # beforeoneday = Data[Data['daystime'] == LabelDay-datetime.timedelta(days=1)]
    beforeonedayuser_count = pd.crosstab(beforeoneday.user_id,beforeoneday.behavior_type)
    countAverage = user_count/FEATURE_EXTRACTION_SLOT
    buyRate = pd.DataFrame()
    buyRate['click'] = user_count[1]/user_count[4]
    buyRate['skim'] = user_count[2]/user_count[4]
    buyRate['collect'] = user_count[3]/user_count[4]
    buyRate.index = user_count.index

    buyRate_2 = pd.DataFrame()
    buyRate_2['click'] = user_count_before5[1]/user_count_before5[4]
    buyRate_2['skim'] = user_count_before5[2]/user_count_before5[4]
    buyRate_2['collect'] = user_count_before5[3]/user_count_before5[4]
    buyRate_2.index = user_count_before5.index

    buyRate_3 = pd.DataFrame()
    buyRate_3['click'] = user_count_before_3[1]/user_count_before_3[4]
    buyRate_3['skim'] = user_count_before_3[2]/user_count_before_3[4]
    buyRate_3['collect'] = user_count_before_3[3]/user_count_before_3[4]
    buyRate_3.index = user_count_before_3.index


    buyRate = buyRate.replace([np.inf, -np.inf], 0)
    buyRate_2 = buyRate_2.replace([np.inf, -np.inf], 0)
    buyRate_3 = buyRate_3.replace([np.inf, -np.inf], 0)

    long_online = pd.pivot_table(beforeoneday,index=['user_id'],values=['hours'],aggfunc=[np.min,np.max,np.ptp])


    user_id_feture = pd.merge(user_count,beforeonedayuser_count,how='left',right_index=True,left_index=True)
    user_id_feture = pd.merge(user_id_feture,countAverage,how='left',right_index=True,left_index=True)
    user_id_feture = pd.merge(user_id_feture,buyRate,how='left',right_index=True,left_index=True)
    user_id_feture = pd.merge(user_id_feture,user_count_before5,how='left',right_index=True,left_index=True)
    user_id_feture = pd.merge(user_id_feture,user_count_before_3,how='left',right_index=True,left_index=True)
    user_id_feture = pd.merge(user_id_feture,user_count_before_2,how='left',right_index=True,left_index=True)
    user_id_feture = pd.merge(user_id_feture,long_online,how='left',right_index=True,left_index=True)
#    user_id_feture = pd.merge(user_id_feture,buyRate_2,how='left',right_index=True,left_index=True)
#    user_id_feture = pd.merge(user_id_feture,buyRate_3,how='left',right_index=True,left_index=True)
    user_id_feture.fillna(0,inplace=True)
    return user_id_feture



def user_item_feture(data,end_time,beforeoneday):   
    # data = Data[(Data['daystime']<LabelDay) & (Data['daystime']>LabelDay-datetime.timedelta(days=FEATURE_EXTRACTION_SLOT))]
    user_item_count = pd.crosstab([data.user_id,data.item_id],data.behavior_type)
    # beforeoneday = Data[Data['daystime'] == LabelDay-datetime.timedelta(days=1)]
    user_item_count_5=None
    if (((end_time-datetime.timedelta(days=5))<datetime.datetime(2014,12,13,0,0,0))&((end_time-datetime.timedelta(days=5))>datetime.datetime(2014,12,10,0,0,0))):
        beforefiveday = data[data['daystime']>=end_time-datetime.timedelta(days=5+2)]
        user_item_count_5 = pd.crosstab([beforefiveday.user_id,beforefiveday.item_id],beforefiveday.behavior_type)
    else:
        beforefiveday = data[data['daystime']>=end_time-datetime.timedelta(days=5)]
        user_item_count_5 = pd.crosstab([beforefiveday.user_id,beforefiveday.item_id],beforefiveday.behavior_type)
    user_item_count_3=None
    if (((end_time-datetime.timedelta(days=5))<datetime.datetime(2014,12,13,0,0,0))&((end_time-datetime.timedelta(days=5))>datetime.datetime(2014,12,10,0,0,0))):
        beforethreeday = data[data['daystime']>=end_time-datetime.timedelta(days=3+2)]
        user_item_count_3 = pd.crosstab([beforethreeday.user_id,beforethreeday.item_id],beforethreeday.behavior_type)
    else:
        beforethreeday = data[data['daystime']>=end_time-datetime.timedelta(days=3)]
        user_item_count_3 = pd.crosstab([beforethreeday.user_id,beforethreeday.item_id],beforethreeday.behavior_type)

    user_item_count_2=None
    if (((end_time-datetime.timedelta(days=5))<datetime.datetime(2014,12,13,0,0,0))&((end_time-datetime.timedelta(days=5))>datetime.datetime(2014,12,10,0,0,0))):
        beforethreeday = data[data['daystime']>=end_time-datetime.timedelta(days=7+2)]
        user_item_count_2 = pd.crosstab([beforethreeday.user_id,beforethreeday.item_id],beforethreeday.behavior_type)
    else:
        beforethreeday = data[data['daystime']>=end_time-datetime.timedelta(days=7)]
        user_item_count_2 = pd.crosstab([beforethreeday.user_id,beforethreeday.item_id],beforethreeday.behavior_type)
        
    beforeonedayuser_item_count = pd.crosstab([beforeoneday.user_id,beforeoneday.item_id],beforeoneday.behavior_type)
    
#    _live = user_item_long_touch(data)
    
    
    max_touchtime = pd.pivot_table(beforeoneday,index=['user_id','item_id'],values=['hours'],aggfunc=[np.min,np.max])
    max_touchtype = pd.pivot_table(beforeoneday,index=['user_id','item_id'],values=['behavior_type'],aggfunc=np.max)
    user_item_feture = pd.merge(user_item_count,beforeonedayuser_item_count,how='left',right_index=True,left_index=True)
    user_item_feture = pd.merge(user_item_feture,max_touchtime,how='left',right_index=True,left_index=True)
    user_item_feture = pd.merge(user_item_feture,max_touchtype,how='left',right_index=True,left_index=True)
#    user_item_feture = pd.merge(user_item_feture,_live,how='left',right_index=True,left_index=True)

    user_item_feture = pd.merge(user_item_feture,user_item_count_5,how='left',right_index=True,left_index=True)
    user_item_feture = pd.merge(user_item_feture,user_item_count_3,how='left',right_index=True,left_index=True)
    user_item_feture = pd.merge(user_item_feture,user_item_count_2,how='left',right_index=True,left_index=True)
    user_item_feture.fillna(0,inplace=True)
    return user_item_feture

def user_cate_feture(data,end_time,beforeoneday):   
    # data = Data[(Data['daystime']<LabelDay) & (Data['daystime']>LabelDay-datetime.timedelta(days=FEATURE_EXTRACTION_SLOT))]
    user_item_count = pd.crosstab([data.user_id,data.item_category],data.behavior_type)
    # beforeoneday = Data[Data['daystime'] == LabelDay-datetime.timedelta(days=1)]
    
    user_cate_count_5=None
    if (((end_time-datetime.timedelta(days=5))<datetime.datetime(2014,12,13,0,0,0))&((end_time-datetime.timedelta(days=5))>datetime.datetime(2014,12,10,0,0,0))):
        beforefiveday = data[data['daystime']>=(end_time-datetime.timedelta(days=5+2))]
        user_cate_count_5 = pd.crosstab([beforefiveday.user_id,beforefiveday.item_category],beforefiveday.behavior_type)
    else:
        beforefiveday = data[data['daystime']>=(end_time-datetime.timedelta(days=5))]
        user_cate_count_5 = pd.crosstab([beforefiveday.user_id,beforefiveday.item_category],beforefiveday.behavior_type)
    user_cate_count_3 = None
    if (((end_time-datetime.timedelta(days=5))<datetime.datetime(2014,12,13,0,0,0))&((end_time-datetime.timedelta(days=5))>datetime.datetime(2014,12,10,0,0,0))):
        beforethreeday = data[data['daystime']>=(end_time-datetime.timedelta(days=3+2))]
        user_cate_count_3 = pd.crosstab([beforethreeday.user_id,beforethreeday.item_category],beforethreeday.behavior_type)
    else:
        beforethreeday = data[data['daystime']>=(end_time-datetime.timedelta(days=3))]
        user_cate_count_3 = pd.crosstab([beforethreeday.user_id,beforethreeday.item_category],beforethreeday.behavior_type)


    user_cate_count_2 = None
    if (((end_time-datetime.timedelta(days=5))<datetime.datetime(2014,12,13,0,0,0))&((end_time-datetime.timedelta(days=5))>datetime.datetime(2014,12,10,0,0,0))):
        beforethreeday = data[data['daystime']>=(end_time-datetime.timedelta(days=7+2))]
        user_cate_count_2 = pd.crosstab([beforethreeday.user_id,beforethreeday.item_category],beforethreeday.behavior_type)
    else:
        beforethreeday = data[data['daystime']>=(end_time-datetime.timedelta(days=7))]
        user_cate_count_2 = pd.crosstab([beforethreeday.user_id,beforethreeday.item_category],beforethreeday.behavior_type)
        
#    _live = user_cate_long_touch(data)
    beforeonedayuser_item_count = pd.crosstab([beforeoneday.user_id,beforeoneday.item_category],beforeoneday.behavior_type)
    max_touchtime = pd.pivot_table(beforeoneday,index=['user_id','item_category'],values=['hours'],aggfunc=[np.min,np.max])
    max_touchtype = pd.pivot_table(beforeoneday,index=['user_id','item_category'],values=['behavior_type'],aggfunc=np.max)
    user_cate_feture = pd.merge(user_item_count,beforeonedayuser_item_count,how='left',right_index=True,left_index=True)
    user_cate_feture = pd.merge(user_cate_feture,max_touchtime,how='left',right_index=True,left_index=True)
    user_cate_feture = pd.merge(user_cate_feture,max_touchtype,how='left',right_index=True,left_index=True)
#    user_cate_feture = pd.merge(user_cate_feture,_live,how='left',right_index=True,left_index=True)
    user_cate_feture = pd.merge(user_cate_feture,user_cate_count_5,how='left',right_index=True,left_index=True)
    user_cate_feture = pd.merge(user_cate_feture,user_cate_count_3,how='left',right_index=True,left_index=True)
    user_cate_feture = pd.merge(user_cate_feture,user_cate_count_2,how='left',right_index=True,left_index=True)
    user_cate_feture.fillna(0,inplace=True)
    return user_cate_feture


In [ ]:
LabelDay = datetime.datetime(2014,12,17,0,0,0)
predict_set_flag = True
#predict_set_flag = False #means get train data set
if __name__ == '__main__':
    result=[]
    feature_u = [[] for i in range(3)]
    ##根据提取特征的时间范围的不同，特征提取窗口的滑动次数也是会相应变化的。
    ##另：以双十二为分界线分别单独提取特征;还是以忽略的方式，有待确定（验证）
    #window_size = 4#提取特征的时间范围，单位是天
    #window_slide_num = 30#特征提取窗口的滑动次数
    window_slide_num = [7,7,7]
    print(LabelDay)
    #for i in list(range(window_size))[1:]:

    for i in [1,3,6]:
        k=1
        print('特征窗口的size：', i, '天')
        if(predict_set_flag):
            LabelDay = datetime.datetime(2014,12,17,0,0,0)
            train_data_set_dir_and_file_name = '../DataSet/'+'predict_data_set'+str(i)+'.csv'
            window_slide_num = [7]
        else:
            LabelDay = datetime.datetime(2014,12,10,0,0,0)
            train_data_set_dir_and_file_name = '../DataSet/'+'train_data_set'+str(i)+'.csv'
        #for j in list(range(window_slide_num)):
        for j in window_slide_num:   
            start_day = (LabelDay - datetime.timedelta(days=i))    
            '''
            #暂时不考虑剔除12.11和12.12两天的数据
            if ((start_day <= datetime.datetime(2014,12,12,0,0,0)) & (LabelDay > datetime.datetime(2014,12,12,0,0,0))):
                #user_behavior_data = Data[(Data['daystime'] >= (LabelDay - datetime.timedelta(days=i))) & (Data['daystime'] < LabelDay)]
                LabelDay = datetime.datetime(2014,12,10,0,0,0)
                start_day = LabelDay - datetime.timedelta(days=i)   
                user_behavior_data = Data[(Data['daystime'] >= start_day) & (Data['daystime'] < LabelDay)]
                label_day_data = Data[Data['daystime']==LabelDay]
                
            else:
                user_behavior_data = Data[(Data['daystime'] >= start_day) & (Data['daystime'] < LabelDay)]
                label_day_data = Data[Data['daystime']==LabelDay]
            '''
            '''
            if(start_day>=datetime.datetime(2014,11,18,0,0,0)):
                user_behavior_data = Data[(Data['daystime'] >= start_day) & (Data['daystime'] < LabelDay)]
                label_day_data = Data[Data['daystime']==LabelDay]
            else:
                print('end')
                break
            '''
            #剔除 12.11和12.12的两天的数据
            if(start_day>=datetime.datetime(2014,11,18,0,0,0)):
                if(start_day == datetime.datetime(2014,12,12,0,0,0) or (start_day == datetime.datetime(2014,12,11,0,0,0))):
                    LabelDay = datetime.datetime(2014,12,10,0,0,0)
                    start_day = (LabelDay - datetime.timedelta(days=i))    
                    print('窗口的位置：[', start_day, LabelDay ,')')
                print('窗口的位置：[', start_day, LabelDay ,')')   
                user_behavior_data = Data[(Data['daystime'] >= start_day) & (Data['daystime'] < LabelDay)]
                label_data_set = Data[(Data['daystime'] >= start_day) & (Data['daystime'] <= LabelDay)]
                #label_day_data = Data[Data['daystime']==LabelDay]
            else:
                print('end')
                break            
            #train_user_window1 = Data[(Data['daystime'] > (LabelDay - datetime.timedelta(days=FEATURE_EXTRACTION_SLOT))) & (Data['daystime'] < LabelDay)]
            #beforeoneday = Data[Data['daystime'] == (LabelDay-datetime.timedelta(days=1))]
            #print(beforeoneday)
            # beforetwoday = Data[(Data['daystime'] >= (LabelDay-datetime.timedelta(days=2))) & (Data['daystime'] < LabelDay)]
            # beforefiveday = Data[(Data['daystime'] >= (LabelDay-datetime.timedelta(days=5))) & (Data['daystime'] < LabelDay)]
            #print(Data.head(5))
            #print(user_behavior_data.head(5))
            #提取特征部分
            #print(user_behavior_data.head(5))
            u = get_feature_u(copy.deepcopy(user_behavior_data))
            i_i  =  get_feature_i(copy.deepcopy(user_behavior_data))
            c =  get_feature_c(copy.deepcopy(user_behavior_data))
            ic = get_feature_ic(copy.deepcopy(user_behavior_data))
            ui = get_feature_ui(copy.deepcopy(user_behavior_data))
            #uc = get_feature_ui(copy.deepcopy(user_behavior_data))
            #print(ui.head(5))
            
            #获取标签日当天的用户购买行为，对a进行打标签
            label_data = get_label_set(copy.deepcopy(label_data_set),LabelDay) 
            #label_data = label_data.head(20)##test
            print('label_data:',len(label_data))
            #label_data.to_csv('../DataSet/label_data.csv',mode='a',index=None)
            #LabelDay = (LabelDay - datetime.timedelta(days=1))    
            LabelDay = (LabelDay - datetime.timedelta(days=j))
                       
            #对数据进行整合，获取train_data_set数据集
            #U
            label_data.set_index(keys=['user_id'], inplace=True)
            label_data.sort_index(inplace=True)
            #print(label_data.head(10))
            u.set_index(keys=['user_id'], inplace=True)
            u.sort_index(inplace=True)
            label_data = label_data.join(u,how='left').fillna(0)
            #print('after join u ,label_data:',len(label_data))
            # I
            label_data.reset_index(inplace=True)
            #print(label_data.head(10))
            label_data.set_index(keys=['item_id'], inplace=True)
            label_data.sort_index(inplace=True)
            i_i.set_index(keys=['item_id'], inplace=True)
            i_i.sort_index(inplace=True)
            label_data = label_data.join(i_i,how='left').fillna(0)
            print('after join i ,label_data:',len(label_data))
            #C
            label_data.reset_index(inplace=True)
            label_data.set_index(keys=['item_category'], inplace=True)
            label_data.sort_index(inplace=True)
            c.set_index(keys=['item_category'], inplace=True)
            c.sort_index(inplace=True)
            label_data = label_data.join(c,how='left').fillna(0)
            print('after join c ,label_data:',len(label_data))
            #IC
            label_data.reset_index(inplace=True)
            label_data.set_index(keys=['item_id','item_category'], inplace=True)
            label_data.sort_index(inplace=True)
            ic.set_index(keys=['item_id','item_category'], inplace=True)
            ic.sort_index(inplace=True)
            label_data = label_data.join(ic,how='left').fillna(0)
            print('after join ic ,label_data:',len(label_data))
            #UI
            label_data.reset_index(inplace=True)
            label_data.set_index(keys=['item_id','user_id'], inplace=True)
            label_data.sort_index(inplace=True)
            ui.set_index(keys=['item_id','user_id'], inplace=True)
            ui.sort_index(inplace=True)
            label_data = label_data.join(ui,how='left').fillna(0)
            print('after join ui ,label_data:',len(label_data))
            #UC
            #waiting to add
            
            #
            label_data.reset_index(inplace=True)
            #print(label_data.head(5))
            if(k==1):
                label_data.to_csv(train_data_set_dir_and_file_name,mode='a',header=True,index=None)
                k=k-1
            else:
                label_data.to_csv(train_data_set_dir_and_file_name,mode='a',header=False,index=None)
        del(label_data)
    print('hello world!')
    if(predict_set_flag):
        train_data_set1 = pd.read_csv("../DataSet/predict_data_set1.csv",header='infer')
        #train_data_set1[['user_id', 'item_id']] = train_data_set1[['user_id', 'item_id']].astype(str)
        print(len(train_data_set1))
    
        train_data_set2 = pd.read_csv("../DataSet/predict_data_set3.csv",header='infer')
        #train_data_set2[['user_id', 'item_id']] = train_data_set2[['user_id', 'item_id']].astype(str)
        print(len(train_data_set2))
        
        train_data_set3 = pd.read_csv("../DataSet/predict_data_set6.csv",header='infer')
        #train_data_set3[['user_id', 'item_id']] = train_data_set3[['user_id', 'item_id']].astype(str)
        print(len(train_data_set3))
    else:
        train_data_set1 = pd.read_csv("../DataSet/train_data_set1.csv",header='infer')
        #train_data_set1[['user_id', 'item_id']] = train_data_set1[['user_id', 'item_id']].astype(str)
        print(len(train_data_set1))
        #print(train_data_set1.head(5))
        train_data_set2 = pd.read_csv("../DataSet/train_data_set3.csv",header='infer')
        #train_data_set2[['user_id', 'item_id']] = train_data_set2[['user_id', 'item_id']].astype(str)
        print(len(train_data_set2))
        #print(train_data_set2.head(5))
       
    train_data_set2.rename(columns={'item_id':'item_id','user_id':'user_id', 'item_category':'2_item_category', 'label':'2_label','u_b1_count':'2_u_b1_count',
                                                 'u_b2_count':'2_u_b2_count', 'u_b3_count':'2_u_b3_count', 'u_b4_count':'2_u_b4_count', 'u_b_count':'2_u_b_count', 'i_b1_count':'2_i_b1_count',
                                                   'i_b2_count':'2_i_b2_count', 'i_b3_count':'2_i_b3_count', 'i_b4_count':'2_i_b4_count', 'i_b_count':'2_i_b_count', 'i_b4_rate':'2_i_b4_rate',
                                                   'i_user_num':'2_i_user_num', 'c_b1_count':'2_c_b1_count', 'c_b2_count':'2_c_b2_count', 'c_b3_count':'2_c_b3_count', 'c_b4_count':'2_c_b4_count',
                                                   'c_b_count':'2_c_b_count', 'c_b4_rate':'2_c_b4_rate', 'c_user_num':'2_c_user_num', 'ic_sales_num':'2_ic_sales_num', 'ic_user_num':'2_ic_user_num',
                                                   'ic_behavior_num':'2_ic_behavior_num', 'ui_sort':'2_ui_sort', 'ui_b1_count':'2_ui_b1_count', 'ui_b2_count':'2_ui_b2_count',
                                                   'ui_b3_count':'2_ui_b3_count', 'ui_b4_count':'2_ui_b4_count', 'ui_cumcount':'2_ui_cumcount', 'ui_b_count':'2_ui_b_count',
                                    'ui_b2_count_in_6':'2_ui_b2_count_in_6',
                                    'ui_b3_count_in_6':'2_ui_b3_count_in_6',
                                    'ui_b4_count_in_6':'2_ui_b4_count_in_6'},inplace=True)
    print(len(train_data_set2.columns))
    print(train_data_set2.columns)
    #temp = pd.concat([train_data_set1,train_data_set2],axis=1).fillna(0)
    temp = train_data_set2.merge(train_data_set1, how='left', on=['item_id','user_id']).fillna(0)
    del(train_data_set1)
    del(train_data_set2)
    train_data_set3 = pd.read_csv("../DataSet/train_data_set6.csv",header='infer')
    #train_data_set3[['user_id', 'item_id']] = train_data_set3[['user_id', 'item_id']].astype(str)
    print(len(train_data_set3))
    #print(train_data_set3.head(5))
    train_data_set3.rename(columns={'item_id':'item_id','user_id':'user_id', 'item_category':'3_item_category', 'label':'3_label','u_b1_count':'3_u_b1_count',
                                                 'u_b2_count':'3_u_b2_count', 'u_b3_count':'3_u_b3_count', 'u_b4_count':'3_u_b4_count', 'u_b_count':'3_u_b_count', 'i_b1_count':'3_i_b1_count',
                                                   'i_b2_count':'3_i_b2_count', 'i_b3_count':'3_i_b3_count', 'i_b4_count':'3_i_b4_count', 'i_b_count':'3_i_b_count', 'i_b4_rate':'3_i_b4_rate',
                                                   'i_user_num':'3_i_user_num', 'c_b1_count':'3_c_b1_count', 'c_b2_count':'3_c_b2_count', 'c_b3_count':'3_c_b3_count', 'c_b4_count':'3_c_b4_count',
                                                   'c_b_count':'3_c_b_count', 'c_b4_rate':'3_c_b4_rate', 'c_user_num':'3_c_user_num', 'ic_sales_num':'3_ic_sales_num', 'ic_user_num':'3_ic_user_num',
                                                   'ic_behavior_num':'3_ic_behavior_num', 'ui_sort':'3_ui_sort', 'ui_b1_count':'3_ui_b1_count', 'ui_b2_count':'3_ui_b2_count',
                                                   'ui_b3_count':'3_ui_b3_count', 'ui_b4_count':'3_ui_b4_count', 'ui_cumcount':'3_ui_cumcount', 'ui_b_count':'3_ui_b_count',
                                    'ui_b2_count_in_6':'3_ui_b2_count_in_6',
                                    'ui_b3_count_in_6':'3_ui_b3_count_in_6',
                                    'ui_b4_count_in_6':'3_ui_b4_count_in_6'},inplace=True)

    print(len(train_data_set3.columns))
    print(train_data_set3.columns)
    #train_data_set = pd.concat([temp,train_data_set3],axis=1).fillna(0)
    train_data_set = train_data_set3.merge(temp, how='left', on=['item_id','user_id']).fillna(0)
    #print('train_data_set\n',train_data_set.head(30))
    del(train_data_set3)
    del(temp)
    if(predict_set_flag):
        train_data_set.drop(train_data_set.columns[[2,33,34,64,65]], axis=1,inplace=True)
        #train_data_set = train_data_set[train_data_set.item_id.isin(list(item_table.item_id))]
        train_data_set.rename(columns={"3_label":"label"},inplace=True)
        train_data_set.to_csv("../DataSet/predict_set.csv",index=None)
    else:
        #train_data_set.to_csv("../DataSet/train_data_set_all_columns_name.csv",index=None)
        train_data_set.drop(train_data_set.columns[[0,1,2,33,34,64,65]], axis=1,inplace=True)
        train_data_set.rename(columns={"3_label":"label"},inplace=True)
        train_data_set.to_csv("../DataSet/train_data_set_all.csv",index=None)
        print("train_data_set column name list:",train_data_set.columns,"train_data_set column name size:",len(train_data_set.columns))
    del(train_data_set)
    print('main function end')